# Tratamento de dados

In [6]:
# Importação de bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd

Utilizaremos os dados dos sites 
- [x] [CBF - Série A](https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a)
- [x] [CBF - Série B](https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b)
- [x] [CBF - Série C](https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-c)

In [7]:
# URLs para web scraping:
url_serie_a = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a'
url_serie_b = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b'
url_serie_c = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-c'

# Fazendo o request para o site:
request_brasileirao_a = requests.get(url_serie_a)
request_brasileirao_b = requests.get(url_serie_b)
request_brasileirao_c = requests.get(url_serie_c)

# Utilizando BeautifulSoup para tratamento HTML:
soup_a = BeautifulSoup(request_brasileirao_a.text, 'html.parser')
soup_b = BeautifulSoup(request_brasileirao_b.text, 'html.parser')
soup_c = BeautifulSoup(request_brasileirao_c.text, 'html.parser')

# Achando as tabelas com as informações:
tabela_a = soup_a.find(class_='table m-b-20 tabela-expandir')
tabela_b = soup_b.find(class_='table m-b-20 tabela-expandir')
tabela_c = soup_c.find(class_='table m-b-20 tabela-expandir')

## Realizando a coleta das tabelas:

As tabelas em HTML são geralmente organizadas da seguinte maneira:

```
<table>
  <tr>
    <th>Header 1</th>
    <th>Header 2</th>
    <th>Header 3</th>
  </tr>
  <tr>
    <td>Alfreds</td>
    <td>Maria</td>
    <td>Germany</td>
  </tr>
  <tr>
    <td>Centro</td>
    <td>Francisco</td>
    <td>Mexico</td>
  </tr>
</table>
```

Contudo, no site da CBF, a tabela está organizada da seguinte maneira:

```
<table>
  <tr>
    <th>Colocção</th>
    <th>Pontos</th>
    <th>Outros Dados</th>
  </tr>
  <tr>
    <td>1</td>
    <th>62</th>
    <td>Germany</td>
  </tr>
  <tr>
    <td>2</td>
    <th>56</th>
    <td>Mexico</td>
  </tr>
</table>
```
Note que existem headers que deverão ser adicionados à tabela e não serem considerados como células do cabeçalho da tebala.


### Veja a diferença:

1 - Headers no cabeçalho:

<table>
  <tr>
    <th>Colocção</th>
    <th>Pontos</th>
    <th>Outros Dados</th>
  </tr>
  <tr>
    <td>1</td>
    <td>62</td>
    <td>Germany</td>
  </tr>
  <tr>
    <td>2</td>
    <td>56</td>
    <td>Mexico</td>
  </tr>
</table>

2- Headers no cabeçalho e na Tabela (Quantidade de pontos):
<table>
  <tr>
    <th>Colocção</th>
    <th>Pontos</th>
    <th>Outros Dados</th>
  </tr>
  <tr>
    <td>1</td>
    <th>62</th>
    <td>Germany</td>
  </tr>
  <tr>
    <td>2</td>
    <th>56</th>
    <td>Mexico</td>
  </tr>
</table>

### Tabela Série A

In [8]:
# ---------------- TABELA SÉRIE A ----------------
# Obtendo o título das colunas e a pontuação com <th>.
classes_e_pontuacao_a = []
for i in tabela_a.find_all('th'):
    title = i.text
    classes_e_pontuacao_a.append(title)

# Separando a categoria da pontuação.
categorias_a = classes_e_pontuacao_a[:-20]
pontos_a = classes_e_pontuacao_a[-20:]

dados_tabela_a = []
# Obtendo todo os dados da tabela com <td>.
for j in tabela_a.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    dados_tabela_a.append(row)

# Separando os dados da tabela das informações adicionais.
tabela_principal = dados_tabela_a[::2]
tabela_info = dados_tabela_a[1::2]

# Tratamento da tabela principal.
categorias_a.insert(1, 'Time')
for dado in tabela_principal[:20]:
    # Colocando Posição e Time
    dado[0] = dado[0].replace('\n', '').replace('\xa00', '').replace('+1', '').replace('-1', '')
    dado[0] = dado[0].replace('+2', '').replace('-2', '').replace('+3', '').replace('-3', '')
    dado.insert(0, dado[0].split('º')[0])
    dado.insert(1, dado[1].split('º')[1])
    dado.pop(2)
    # Jogos Recentes
    dado[12] = dado[12][1:-1].replace('\n', ',')

# Colocando a pontuação na tabela:
for indice, pontuacao in enumerate(pontos_a):
    tabela_principal[indice].insert(2, pontuacao)

# Criando DataFrame da tabela.
DadosBrasileirao_a = pd.DataFrame(columns=categorias_a, data=tabela_principal[:20])
print(DadosBrasileirao_a)


   Posição                       Time PTS   J   V   E   D  GP  GC   SG   CA  \
0        1      Atlético Mineiro - MG  84  38  26   6   6  67  34   33   82   
1        2              Flamengo - RJ  71  38  21   8   9  69  36   33   82   
2        3             Palmeiras - SP  66  38  20   6  12  58  43   15   87   
3        4             Fortaleza - CE  58  38  17   7  14  44  45   -1   95   
4        5           Corinthians - SP  57  38  15  12  11  40  36    4   61   
5        6   Red Bull Bragantino - SP  56  38  14  14  10  55  46    9   95   
6        7            Fluminense - RJ  54  38  15   9  14  38  38    0   91   
7        8               America - MG  53  38  13  14  11  41  37    4   66   
8        9              Atlético - GO  53  38  13  14  11  33  36   -3   98   
9       10                Santos - SP  50  38  12  14  12  35  40   -5  103   
10      11                 Ceará - CE  50  38  11  17  10  39  38    1   90   
11      12         Internacional - RS  48  38  12  1

### Tabela Série B

In [9]:
# ---------------- TABELA SÉRIE B ----------------
# Obtendo o título das colunas e a pontuação com <th>
classes_e_pontuacao_b = []
for i in tabela_b.find_all('th'):
    title = i.text
    classes_e_pontuacao_b.append(title)

# Separando a categoria da pontuação.
categorias_b = classes_e_pontuacao_b[:-20]
pontos_b = classes_e_pontuacao_b[-20:]

dados_tabela_b = []
# Obtendo todo os dados da tabela com <td>.
for j in tabela_b.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    dados_tabela_b.append(row)

tabela_principal = dados_tabela_b[::2]
tabela_info = dados_tabela_b[1::2]

# Tratamentoda Posição e Time
categorias_b.insert(1, 'Time')
for dado in tabela_principal[:20]:
    # Colocando Posição e Time
    dado[0] = dado[0].replace('\n', '').replace('\xa00', '').replace('+1', '').replace('-1', '')
    dado[0] = dado[0].replace('+2', '').replace('-2', '').replace('+3', '').replace('-3', '')
    dado.insert(0, dado[0].split('º')[0])
    dado.insert(1, dado[1].split('º')[1])
    dado.pop(2)
    # Jogos Recentes
    dado[12] = dado[12][1:-1].replace('\n', ',')

# Colocando a pontuação na tabela:
for indice, pontuacao in enumerate(pontos_b):
    tabela_principal[indice].insert(2, pontuacao)

# Criando DataFrame da tabela.
DadosBrasileirao_b = pd.DataFrame(columns=categorias_b, data=tabela_principal[:20])
print(DadosBrasileirao_b)

   Posição                 Time PTS   J   V   E   D  GP  GC   SG   CA  CV   %  \
0        1        Botafogo - RJ  70  38  20  10   8  56  31   25   71   5  61   
1        2           Goiás - GO  65  38  17  14   7  48  31   17   96   4  57   
2        3        Coritiba - PR  64  38  18  10  10  49  35   14   99   3  56   
3        4            Avaí - SC  64  38  18  10  10  44  35    9   92   6  56   
4        5             Csa - AL  62  38  18   8  12  48  33   15   99   3  54   
5        6         Guarani - SP  60  38  16  12  10  54  41   13  106  12  52   
6        7             Crb - AL  60  38  16  12  10  47  39    8   84   5  52   
7        8         Náutico - PE  53  38  14  11  13  50  50    0  103  11  46   
8        9       Vila Nova - GO  51  38  12  15  11  35  36   -1   95   3  44   
9       10   Vasco da Gama - RJ  49  38  13  10  15  43  52   -9   94   9  42   
10      11     Ponte Preta - SP  49  38  12  13  13  39  40   -1   79   7  42   
11      12        Operário -

### Tabela Série C

In [10]:
# ---------------- TABELA SÉRIE C ----------------
# Obtendo o título das colunas e a pontuação com <th>.
classes_e_pontuacao_c = []
for i in tabela_c.find_all('th'):
    title = i.text
    classes_e_pontuacao_c.append(title)

# Separando a categoria da pontuação.
categorias_c = classes_e_pontuacao_c[:-20]
pontos_c = classes_e_pontuacao_c[20:]

dados_tabela_c = []
# Obtendo todo os dados da tabela com <td>.
for j in tabela_c.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    dados_tabela_c.append(row)

tabela_principal = dados_tabela_c[::2]
tabela_info = dados_tabela_c[1::2]

# Tratamentoda Posição e Time
categorias_a.insert(1, 'Time')
for dado in tabela_principal[:20]:
    # Colocando Posição e Time
    dado[0] = dado[0].replace('\n', '').replace('\xa00', '').replace('+1', '').replace('-1', '')
    dado[0] = dado[0].replace('+2', '').replace('-2', '').replace('+3', '').replace('-3', '')
    dado.insert(0, dado[0].split('º')[0])
    dado.insert(1, dado[1].split('º')[1])
    dado.pop(2)
    # Jogos Recentes
    dado[12] = dado[12][1:-1].replace('\n', ',')

# Colocando a pontuação na tabela:
for indice, pontuacao in enumerate(pontos_b):
    tabela_principal[indice].insert(2, pontuacao)

# Criando DataFrame da tabela.
DadosBrasileirao_c = pd.DataFrame(columns=categorias_c, data=tabela_principal[:20])
print(DadosBrasileirao_c)

IndexError: list index out of range